# Labb 1 (vecka 1): Välja rätt metric och tunea Ridge

Ni kommer att få 3 olika scenarion att utforska

## Mål
Ni ska träna en Ridge-regressionsmodell och:
1) välja **vilken utvärderings-metric som är mest rimlig** i ett givet scenario. 
2) **optimera (tunea) modellen** mot den metricen genom att skapa ett bra grid för hyperparametern **alpha**.

> Fokus i den här labben är inte att skriva mycket kod från scratch,
> utan att förstå **vad koden gör**, **varför vi gör stegen** och **hur vi tolkar resultaten**.

---

## Scenarion
Ni ska börja med att läsa igenom alla tre scenarion tillsammans och diskutera kort det som står under "Diskutera"

Ni får samma dataset och samma modelltyp – men “vad som är ett bra fel” beror på situationen.

---

### Scenario A: Sjukhus – planera bemanning
Ni förutspår **antal patienter** som kommer in nästa timme/dygn.

- Om modellen ibland **underskattar rejält** → personal räcker inte, köerna växer, stress och risk ökar.
- Om modellen **överskattar lite** → ofta “bara” ineffektivitet (men mindre akut än brist).
- Små fel kan ni leva med – men **enstaka stora fel** kan bli riktigt dyra.

**Diskutera:**
Vilken metric passar bäst om vi vill **straffa stora missar extra mycket**?
Motivera utifrån vad som är “dyrt” i scenariot.

---

### Scenario B: Butik – beställa varor
Ni förutspår efterfrågan på en vara.

- Ibland finns outliers (kampanjer, helger, väder, TikTok-trend) som gör att vissa dagar blir extremt annorlunda.
- Ni vill att modellen ska vara bra på vanliga dagar, inte att den “optimerar” för att passa de få extremdagarna perfekt.
- Tänk så här: vissa metrics överreagerar på stora fel. Om ett enstaka fel blir jättestort kan det dominera hela måttet och styra modellen för mycket.
- I en butik kan det ofta vara bättre att modellen är stabil: hellre “okej” nästan varje dag än att den blir sämre på det normala bara för att jaga extrema toppar.

**Diskutera:**
Vilken metric passar bäst om vi vill vara mindre känsliga för enstaka extrema dagar?
Motivera varför utifrån hur ni tror metricen “straffar” fel.

---

### Scenario C: Rapportering – förklara modellens nytta för en icke-teknisk chef
Ni ska presentera modellen för en chef som inte bryr sig om ML-detaljer.

- Chefen frågar: “Hur mycket bättre är det här än att bara gissa ett standardvärde?”
- En siffra som kan tolkas som **”hur mycket av variationen vi förklarar”** kan vara pedagogisk.
- Ni vill ha något som funkar bra i en presentation, även om man ibland behöver förklara begränsningar.

**Diskutera:**
Vilken metric passar bäst som en **förklarings-/jämförelse-siffra**?
Vad är bra med den – och vad kan vara missvisande?

---

## Uppgifter
1. Diskutera alla tre scenarion kort i gruppen, och välj sedan ett scenario att gå vidare med.
2. Kör notebooken fram till GridSearch-delen block för block och diskutera/tolka output tillsammans.
3. Välj **primär metric** (MAE / RMSE / R²) utifrån ert scenario och skriv en kort motivering.
4. Bygg ert **alpha-grid** (lista med testvärden).
   - Poängen är att experimentera och se hur resultatet ändras.
   - Testa gärna både små och stora alpha.
5. Kör **GridSearchCV** och jämför:
   - bästa alpha
   - CV-resultat (kom ihåg att vissa scoring i scikit-learn blir “negativa”)
   - testresultat (MAE, RMSE, R²)
6. Diskutera i grupp:
   - Varför valde ni just den metricen?
   - Blev bästa alpha annorlunda om ni bytte metric? Varför kan det bli så?

---

Skulle ni hinna klart och har tid över. Testa då ett annat scenario

> ## Leverans (presentera för klassen)
> - Vi optimerade mot: (MAE/RMSE/R²)
> - Varför: 2–3 meningar kopplat till scenario
> - Bästa alpha: …
> - Vad blev er metric på test-data?
> - Om ni skulle gå vidare i verkligheten: 1 sak ni skulle testa (t.ex. fler features, annan modell, mer data, annan metric, större grid)


In [ ]:
# python -m pip install numpy pandas matplotlib seaborn scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)


## Skapa ett dataset

In [ ]:
def generate_demand_dataset(n_days=800, outlier_fraction=0.03, random_seed=42):
    rng_local = np.random.default_rng(random_seed)

    weekday = rng_local.integers(0, 7, size=n_days)
    marketing_spend = rng_local.gamma(shape=2.0, scale=50.0, size=n_days)
    is_campaign = rng_local.binomial(1, 0.25, size=n_days)
    competitor_discount = rng_local.binomial(1, 0.15, size=n_days)
    temperature = rng_local.normal(loc=12, scale=8, size=n_days)

    base = 120
    weekday_effect = np.where(weekday < 5, 10, -15)
    campaign_effect = is_campaign * 45
    competitor_effect = competitor_discount * (-25)
    marketing_effect = 0.35 * marketing_spend
    temp_effect = 1.5 * temperature

    noise = rng_local.normal(0, 20, size=n_days)
    demand = base + weekday_effect + campaign_effect + competitor_effect + marketing_effect + temp_effect + noise

    # outliers
    n_outliers = int(n_days * outlier_fraction)
    outlier_indices = rng_local.choice(n_days, size=n_outliers, replace=False)
    shock = rng_local.normal(loc=220, scale=60, size=n_outliers)
    demand[outlier_indices] += shock

    df = pd.DataFrame({
        "marketing_spend": marketing_spend,
        "weekday": weekday,
        "is_campaign": is_campaign,
        "competitor_discount": competitor_discount,
        "temperature": temperature,
        "demand": demand
    })

    # lite missing values
    missing_mask = rng_local.random(df.shape[0]) < 0.02
    df.loc[missing_mask, "temperature"] = np.nan

    return df

df = generate_demand_dataset()
df.head(10)


## EDA

In [ ]:
print(df.shape)
df.describe(include="all")

In [ ]:
plt.figure()
plt.hist(df["demand"], bins=40)
plt.title("Histogram: demand (kolla outliers)")
plt.xlabel("demand")
plt.ylabel("count")
plt.show()

In [ ]:
plt.figure()
plt.scatter(df["marketing_spend"], df["demand"], alpha=0.5)
plt.title("Scatter: marketing_spend vs demand")
plt.xlabel("marketing_spend")
plt.ylabel("demand")
plt.show()

In [ ]:
corr = df.corr(numeric_only=True)
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(corr, annot=True, ax=ax)
plt.show()

## Train / Test split

In [ ]:
X = df.drop(columns=["demand"])
y = df["demand"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED
)

print("Train:", X_train.shape, "Test:", X_test.shape)


## Baseline

In [ ]:
# Egen baseline: gissa alltid train-medelvärdet
baseline_value = y_train.mean()
y_pred_baseline = np.full(len(y_test), baseline_value)

baseline_mae = mean_absolute_error(y_test, y_pred_baseline)
baseline_rmse = root_mean_squared_error(y_test, y_pred_baseline)
baseline_r2 = r2_score(y_test, y_pred_baseline)

print("Egen baseline (train-mean)")
print(f"MAE:  {baseline_mae:.2f}")
print(f"RMSE: {baseline_rmse:.2f}")
print(f"R²:   {baseline_r2:.3f}")


## Jämför modeller med CV

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

models = {
    "LinearRegression": LinearRegression(),
    "Ridge(alpha=1.0)": Ridge(alpha=1.0, random_state=RANDOM_SEED)
}

scoring = {
    "mae": "neg_mean_absolute_error",
    "rmse": "neg_root_mean_squared_error",
    "r2": "r2"
}

rows = []
for name, model in models.items():
    pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", model)
    ])

    out = cross_validate(pipe, X_train, y_train, cv=cv, scoring=scoring)
    rows.append({
        "model": name,
        "CV_MAE": -out["test_mae"].mean(),
        "CV_RMSE": -out["test_rmse"].mean(),
        "CV_R2": out["test_r2"].mean()
    })

pd.DataFrame(rows)


## Er Uppgift: välj primär metric

In [ ]:
# Välj EN primär metric baserat på ert scenario.
# Skriv exakt en av: "mae"  "rmse"  "r2"
PRIMARY_METRIC = "mae"  # <-- ÄNDRA här

allowed_metrics = {"mae", "rmse", "r2"}
if PRIMARY_METRIC not in allowed_metrics:
    raise ValueError(f"PRIMARY_METRIC måste vara en av {allowed_metrics}. Du skrev: {PRIMARY_METRIC}")

PRIMARY_METRIC

## GridSearchCV på Ridge alpha
om scoring i GridSearchCV (negativa värden)  
GridSearchCV vill alltid MAXIMERA ett score.

För MAE/RMSE är lägre bättre i verkligheten, därför använder scikit-learn en negativ variant:
- neg_mean_absolute_error
- neg_root_mean_squared_error

Det betyder: **högre (mindre negativt) är bättre**
Exempel: **-23 är bättre än -30** (för -23 ligger närmare 0 → mindre fel).  


> **Målet här är att optimera modellen så mycket som möjligt:**  
> Starta grovt → förfina runt bästa alpha

In [ ]:
scoring_map = {
    # scikit-learn använder ofta 'neg_' för loss-mått så att "större är bättre" i GridSearch.
    # Det betyder: när vi maximerar ett negativt tal så minimerar vi egentligen felet.
    "mae": "neg_mean_absolute_error",
    "rmse": "neg_root_mean_squared_error",
    "r2": "r2",
}

# Pipeline = en kedja av steg som alltid körs i samma ordning.
# 1) imputer: fyller ev. saknade värden (NaN) med medianen (robust mot outliers).
# 2) scaler: standardiserar features (viktigt för Ridge/linjära modeller).
# 3) model: själva Ridge-regressionen.
ridge_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("model", Ridge(random_state=RANDOM_SEED)),
])

# =============================
# TODO (NI FYLLER I): alpha-grid
# =============================
# alpha styr hur stark regularisering vi har.
# - Liten alpha => mer flexibel modell (kan passa träningsdata mer).
# - Stor alpha  => mer "straff" => koefficienter pressas mot 0 => enklare modell.
#
# Tips:
# - Testa gärna värden som spänner över flera storleksordningar.
# - Ni kan skriva en lista manuellt eller använda t.ex. np.logspace(...)
ALPHA_GRID = [
    # TODO: Lägg in alpha-värden att testa (flera storleksordningar)
    # Exempel på format: 0.001, 0.01, 0.1, 1.0, 10.0
]

if len(ALPHA_GRID) == 0:
    raise ValueError("ALPHA_GRID är tom. Fyll i en lista med alpha-värden att testa.")

param_grid = {
    "model__alpha": ALPHA_GRID
}

grid = GridSearchCV(
    ridge_pipe,
    param_grid=param_grid,
    scoring=scoring_map[PRIMARY_METRIC],
    cv=cv,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best alpha:", grid.best_params_)
print(f"Best CV score ({PRIMARY_METRIC}):", grid.best_score_, "(notera: neg om MAE/RMSE)")



## Sluttest på test-setet

In [ ]:
best_model = grid.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred)
test_rmse = root_mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print("TEST result (bästa tuned Ridge)")
print(f"MAE:  {test_mae:.2f}")
print(f"RMSE: {test_rmse:.2f}")
print(f"R²:   {test_r2:.3f}")
print("Primär metric:", PRIMARY_METRIC)


## Förbättring mot baseline

In [ ]:
print("Förbättring mot baseline (positivt = bättre)")
print(f"Δ MAE:  {baseline_mae - test_mae:.2f}")
print(f"Δ RMSE: {baseline_rmse - test_rmse:.2f}")
print(f"Δ R²:   {test_r2 - baseline_r2:.3f}")